In [1]:
import numpy as np
import pandas as pd

In [2]:
att_close = pd.read_csv("PA_close_new.csv")
att_open = pd.read_csv("PA_open_new.csv")
print(att_close.shape)
print(att_open.shape)

(226, 57)
(279, 58)


In [3]:
li = ['business_id', 'latitude', 'longitude', 'RestaurantsPriceRange2']
new_close = att_close.loc[:,li]
new_open = att_open.loc[:,li]

In [4]:
df_combined = pd.concat([new_close, new_open])
df_combined.shape

(505, 4)

In [5]:
df_combined.head

<bound method NDFrame.head of                 business_id   latitude  longitude  RestaurantsPriceRange2
0    MTSW4McQd7CbVtyjqoe9mw  39.955505 -75.155564                     2.0
1    MyE_zdul_JO-dOHOug4GQQ  40.119713 -75.009710                     2.0
2    JomDoQafikclSVVCPkWjnA  39.947960 -75.221980                     1.0
3    EraDGJAHjf5IaA9VBGepfg  40.012014 -75.189067                     1.0
4    0346ZLBXCnsGief7r2kmQQ  39.962145 -75.154230                     2.0
..                      ...        ...        ...                     ...
274  hUmHwBQtGg0iH-PZakQJDw  39.971709 -75.135062                     3.0
275  y8gjlpJA89qDRCLC0JQaew  39.950500 -75.166696                     2.0
276  oZzN706lKoL4faaTK739xA  39.940324 -75.149300                     4.0
277  bmJgvpuf2GBXd62ELK0Q2w  39.967551 -75.177122                     1.0
278  wVxXRFf10zTTAs11nr4xeA  40.032483 -75.214430                     2.0

[505 rows x 4 columns]>

In [6]:
longitude_1 = 39.92
longitude_2 = 39.97
latitude_1 = -75.22
latitude_2 = -75.11

# def in_center(lo, la, price ):

#     if (la > longitude_1 and la < longitude_2  and lo > latitude_1 and lo < latitude_2):
#         if  price >2.5:
#         return True
#     else:
#         return False

In [7]:
np.unique(att_close.RestaurantsPriceRange2)

array([ 1.,  2.,  3.,  4., nan])

In [8]:
center_high = []
center_low = []
nocen_high = []
nocen_low = []
df_combined = df_combined.dropna(subset=['RestaurantsPriceRange2'])
for i in range(np.shape(df_combined)[0]):
    lo = df_combined.iloc[i,2]
    la = df_combined.iloc[i,1]
    if (la > longitude_1 and la < longitude_2  and lo > latitude_1 and lo < latitude_2):
        if  df_combined.iloc[i,3] >2.5:
            center_high.append(df_combined.iloc[i,0])
        else:
            center_low.append(df_combined.iloc[i,0])
            
    else:
        if  df_combined.iloc[i,3] >2.5:
            nocen_high.append(df_combined.iloc[i,0])
        else:
            nocen_low.append(df_combined.iloc[i,0])
            
            

In [9]:
## read in review

In [10]:
import json
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from collections import Counter

In [11]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\THINK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\THINK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
data_re= []

with open('review.json', 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        data_re.append(data)

review = pd.DataFrame(data_re)

center_high = center_high.tolist()
center_low = center_low.tolist()
nocen_high = nocen_high.tolist()
nocen_low = nocen_low.tolist()

In [13]:
review_business_id = review["business_id"]
review_ch = []
review_cl = []
review_nh = []
review_nl = []



for i in range(len(review_business_id)):
    if review_business_id[i] in center_high:
        review_ch.append(i)
    elif review_business_id[i] in center_low:
        review_cl.append(i)
    elif review_business_id[i] in nocen_high:
        review_nh.append(i)
    elif review_business_id[i] in nocen_low:
        review_nl.append(i)

In [14]:
PA_ch = review.iloc[review_ch,]
PA_cl = review.iloc[review_cl,]
PA_nh = review.iloc[review_nh,]
PA_nl = review.iloc[review_nl,]

In [17]:
english_stopwords = stopwords.words('english')

def nlp_text(dataframe):
    dataframe["text"] = dataframe["text"].str.lower()  # lower case
    dataframe["text"] = dataframe["text"].str.replace('[^\w\s]', '') # drop off the Punctuation mark
    
    # create the token
    token = dataframe["text"].apply(nltk.word_tokenize)
    
    stop = set(stopwords.words('english'))
    result = token.apply(lambda x: [item for item in x if item not in stop])
    return result


#### PA_ch

In [15]:
PA_ch_1 = PA_ch[PA_ch['stars'] == 1.0]
PA_ch_2 = PA_ch[PA_ch['stars'] == 2.0]
PA_ch_3 = PA_ch[PA_ch['stars'] == 3.0]
PA_ch_4 = PA_ch[PA_ch['stars'] == 4.0]
PA_ch_5 = PA_ch[PA_ch['stars'] == 5.0]
PA_ch_star = [PA_ch_1, PA_ch_2, PA_ch_3,PA_ch_4 ,PA_ch_5]

PA_ch_words = {}


for i, df in enumerate(PA_ch_star):
    all_words = [word for sublist in nlp_text(df) for word in sublist]
    word_counts = Counter(all_words)
    PA_ch_words[f'star{i+1}'] = word_counts 

In [18]:
PA_ch_words = {}


for i, df in enumerate(PA_ch_star):
    all_words = [word for sublist in nlp_text(df) for word in sublist]
    word_counts = Counter(all_words)
    PA_ch_words[f'star{i+1}'] = word_counts 


C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["text"] = dataframe["text"].str.lower()  # lower case
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe["text"] = dataframe["text"].str.replace('[^\w\s]', '') # drop off the Punctuation mark
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [32]:
PA_ch_df = pd.DataFrame.from_dict(PA_ch_words, orient='index').fillna(0).T

In [33]:
PA_ch_df.shape

(25852, 5)

#### PA_cl

In [29]:
PA_cl_1 = PA_cl[PA_cl['stars'] == 1.0]
PA_cl_2 = PA_cl[PA_cl['stars'] == 2.0]
PA_cl_3 = PA_cl[PA_cl['stars'] == 3.0]
PA_cl_4 = PA_cl[PA_cl['stars'] == 4.0]
PA_cl_5 = PA_cl[PA_cl['stars'] == 5.0]
PA_cl_star = [PA_cl_1, PA_cl_2, PA_cl_3,PA_cl_4 ,PA_cl_5]

PA_cl_words = {}


for i, df in enumerate(PA_cl_star):
    all_words = [word for sublist in nlp_text(df) for word in sublist]
    word_counts = Counter(all_words)
    PA_cl_words[f'star{i+1}'] = word_counts 
    
    
PA_cl_df = pd.DataFrame.from_dict(PA_cl_words, orient='index').fillna(0).T

C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["text"] = dataframe["text"].str.lower()  # lower case
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe["text"] = dataframe["text"].str.replace('[^\w\s]', '') # drop off the Punctuation mark
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [31]:
PA_cl_df.shape

(62932, 5)

#### PA_nh

In [37]:
PA_nh_1 = PA_nh[PA_nh['stars'] == 1.0]
PA_nh_2 = PA_nh[PA_nh['stars'] == 2.0]
PA_nh_3 = PA_nh[PA_nh['stars'] == 3.0]
PA_nh_4 = PA_nh[PA_nh['stars'] == 4.0]
PA_nh_5 = PA_nh[PA_nh['stars'] == 5.0]
PA_nh_star = [PA_nh_1, PA_nh_2, PA_nh_3,PA_nh_4 ,PA_nh_5]

from collections import Counter

PA_nh_words = {}


for i, df in enumerate(PA_nh_star):
    all_words = [word for sublist in nlp_text(df) for word in sublist]
    word_counts = Counter(all_words)
    PA_nh_words[f'star{i+1}'] = word_counts 
    
    
PA_nh_df = pd.DataFrame.from_dict(PA_nh_words, orient='index').fillna(0).T

C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["text"] = dataframe["text"].str.lower()  # lower case
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe["text"] = dataframe["text"].str.replace('[^\w\s]', '') # drop off the Punctuation mark
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [38]:
PA_nh_df.shape

(10911, 5)

#### PA_nl

In [34]:
PA_nl_1 = PA_nl[PA_nl['stars'] == 1.0]
PA_nl_2 = PA_nl[PA_nl['stars'] == 2.0]
PA_nl_3 = PA_nl[PA_nl['stars'] == 3.0]
PA_nl_4 = PA_nl[PA_nl['stars'] == 4.0]
PA_nl_5 = PA_nl[PA_nl['stars'] == 5.0]
PA_nl_star = [PA_nl_1, PA_nl_2, PA_nl_3,PA_nl_4 ,PA_nl_5]

from collections import Counter

PA_nl_words = {}


for i, df in enumerate(PA_nl_star):
    all_words = [word for sublist in nlp_text(df) for word in sublist]
    word_counts = Counter(all_words)
    PA_nl_words[f'star{i+1}'] = word_counts 
    
    
PA_nl_df = pd.DataFrame.from_dict(PA_nl_words, orient='index').fillna(0).T

C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["text"] = dataframe["text"].str.lower()  # lower case
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe["text"] = dataframe["text"].str.replace('[^\w\s]', '') # drop off the Punctuation mark
C:\Users\THINK\AppData\Local\Temp\ipykernel_10348\1937035396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [35]:
PA_nl_df.shape

(24389, 5)

#### write into csv

In [39]:
PA_ch_df.to_csv('PA_ch_df.csv', index = True)
PA_cl_df.to_csv('PA_cl_df.csv', index = True)
PA_nh_df.to_csv('PA_nh_df.csv', index = True)
PA_nl_df.to_csv('PA_nl_df.csv', index = True)